In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

# Parsing WIKIPEDIA

In [2]:
cities = []
countries = []
populations = []
timezones = []

In [3]:
links = [
    "https://en.wikipedia.org/wiki/Paris",
    "https://en.wikipedia.org/wiki/Stuttgart",
]

In [4]:
pop_names = ["Total", "City", "Urban", "Metro"]
for link in links:
    response = requests.get(link)
    soup = BeautifulSoup(response.content, "html.parser")
    cities.append(soup.select("span.mw-page-title-main")[0].get_text())
    rows = soup.find_all('tr', class_='mergedtoprow')
    for row in rows:
        label_cell = row.find('th', class_='infobox-label')
        if label_cell and label_cell.get_text(strip=True) == 'Country':
            countries.append(row.find('td', class_='infobox-data').get_text(strip=True))        
            break
    for row in rows:
        label_cell = row.find('th', class_='infobox-label')
        if label_cell and label_cell.get_text(strip=True) == 'Time zone':
            timezones.append(row.find('td', class_='infobox-data').get_text(strip=True))
    merged_rows = soup.find_all('tr', class_='mergedrow')
    test_name = 0
    test_pop = 0   
    for row in merged_rows:
        label_cell = row.find('th', class_='infobox-label')
        for check in pop_names:
            if label_cell and check in label_cell.get_text(strip=True) and test == 0:
                test_name = 1
                break
        if label_cell and test == 1 and test_pop == 0:
            temp = row.find('td', class_='infobox-data').get_text(strip=True)
            if temp.find("(") < 0:
                population = temp.replace(",", "")
                population = re.sub("\[\d+\]", "", population)
                populations.append(population)
                test_pop = 1


In [5]:
cities_df = pd.DataFrame(
    {"city": cities,
     "country": countries,
     "population": populations,
     "timezone": timezones
    }
)

In [6]:
cities_df["population"] = pd.to_numeric(cities_df["population"])

In [7]:
cities_df

,city,country,population,timezone
0,Paris,France,10858852,UTC+01:00(CET)
1,Stuttgart,Germany,626275,UTC+01:00(CET)


In [8]:
cities_df.to_csv("cities.csv", index=False)